# Tutorial 4 - Advanced Workflow - Test Your Own Custom Connector

**Scenario**: 

You are a tester and you want to evaluate your Math application that uses Langchain.<br>
In this case, the existing list of connectors in Moonshot is unable to communicate to this custom Math application.

How do we create a custom connector to use Moonshot to test this custom application?

In this tutorial, you will learn how to:

- Create your own `connector` in Moonshot
- Create a new `connector endpoint` in Moonshot
- Run a new `recipe` on this custom connector

Prerequisite:

1. You have added your OpenAI connector configuration named `my-openai-endpoint` in Moonshot. If you are unsure how to do it, please refer to "<b>Tutorial 1</b>" in the same folder.

**Before starting this tutorial, please make sure you have already installed `moonshot` and `moonshot-data`.**<br>
Otherwise, please refer to "<b>Moonshot - Pre-Req - Setup.ipynb</b>" to install and configure Moonshot first.

## Import and configure Moonshot

In this section, we prepare our Jupyter notebook environment by importing necessary libraries required to execute red teaming session.

> ⚠️ **Note:** Check that `moonshot_data_path` below matches the location where you installed `moonshot-data` and edit the code to match your location if needed.

In [9]:
# Python built-ins:
import os
import sys
import json

# IF you're running this notebook from the moonshot/examples/jupyter-notebook folder, the below
# line will enable you to import moonshot from the local source code. If you installed moonshot
# from pip, you can remove this:
sys.path.insert(0, '../../')

from moonshot.api import (
    api_create_endpoint,
    api_create_recipe,
    api_load_runner,
    api_set_environment_variables
)

# Environment Configuration
# Here we set up the environment variables for the Moonshot framework.
# These variables define the paths to various modules and components used by Moonshot,
# organizing the framework's structure and access points.

# modify moonshot_data_path to point to your own copy of moonshot-data
moonshot_data_path = "./moonshot-data"
env = {
    "ATTACK_MODULES": os.path.join(moonshot_data_path, "attack-modules"),
    "BOOKMARKS": os.path.join(moonshot_data_path, "generated-outputs/bookmarks"),
    "CONNECTORS": os.path.join(moonshot_data_path, "connectors"),
    "CONNECTORS_ENDPOINTS": os.path.join(moonshot_data_path, "connectors-endpoints"),
    "CONTEXT_STRATEGY": os.path.join(moonshot_data_path, "context-strategy"),
    "COOKBOOKS": os.path.join(moonshot_data_path, "cookbooks"),
    "DATABASES": os.path.join(moonshot_data_path, "generated-outputs/databases"),
    "DATABASES_MODULES": os.path.join(moonshot_data_path, "databases-modules"),
    "DATASETS": os.path.join(moonshot_data_path, "datasets"),
    "IO_MODULES": os.path.join(moonshot_data_path, "io-modules"),
    "METRICS": os.path.join(moonshot_data_path, "metrics"),
    "PROMPT_TEMPLATES": os.path.join(moonshot_data_path, "prompt-templates"),
    "RECIPES": os.path.join(moonshot_data_path, "recipes"),
    "RESULTS": os.path.join(moonshot_data_path, "generated-outputs/results"),
    "RESULTS_MODULES": os.path.join(moonshot_data_path, "results-modules"),
    "RUNNERS": os.path.join(moonshot_data_path, "generated-outputs/runners"),
    "RUNNERS_MODULES": os.path.join(moonshot_data_path, "runners-modules"),
}

# Check user has set moonshot_data_path correctly:
if not os.path.isdir(env["ATTACK_MODULES"]):
    raise ValueError(
        "Configured path %s does not exist. Is moonshot-data installed at %s?"
        % (env["ATTACK_MODULES"], moonshot_data_path)
    )

# Apply the environment variables to configure the Moonshot framework.
api_set_environment_variables(env)

# Note: there might be some warning on IProgress not found. we can ignore it for now.

## Create Custom Connector 

In this section, we will learn how to create a custom `connector` and a `connector endpoint` to communicate to a custom Math application. We will use Langchain Agent and OpenAI to create this application.

A Langchain Agent can contain one or more `Tool`. In our application, we will create two tools:

1) Math Tool using `LLMMathChain`
2) Assistant using `LLMChain`

Our assistant will use our Math Tool to answer math questions. This assistant will reply in the following format:

`{"input": <prompt>, "output": <response>}`

### Install Requirements

In [10]:
# Install langchain library
!pip install langchain langchain_openai numexpr

### Custom Connector Code

1. Copy the connector to `./{moonshot_data_path}/connectors/custom-app.py`, <br>
where `{moonshot_data_path}` is the path of your own copy of moonshot-data you specified in the first cell

In [11]:
# Copy the custom-app.py from assets to moonshot-data
import shutil

source_path = f"{moonshot_data_path}/../assets/jupyter-assets-custom-app.py"
destination_path = f"{moonshot_data_path}/connectors/custom-app.py"

shutil.copyfile(source_path, destination_path)
print(f"Copied {source_path} to {destination_path}")


Copied ./moonshot-data/../assets/jupyter-assets-custom-app.py to ./moonshot-data/connectors/custom-app.py


2. Display the contents of this custom connector.

In [12]:
# Display the contents of this custom connector
with open(destination_path, 'r') as file:
    custom_app_contents = file.read()
    print(custom_app_contents)

import os
from typing import Any

from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.chains import LLMChain, LLMMathChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

from moonshot.src.connectors.connector import Connector, perform_retry
from moonshot.src.connectors.connector_response import ConnectorResponse
from moonshot.src.connectors_endpoints.connector_endpoint_arguments import (
    ConnectorEndpointArguments,
)


class MathApplicationConnector(Connector):
    def __init__(self, ep_arguments: ConnectorEndpointArguments):
        """
        Initialize the MathApplicationConnector with endpoint arguments.

        Args:
            ep_arguments (ConnectorEndpointArguments): The endpoint arguments for the connector.
        """
        # Initialize super class
        super().__init__(ep_arguments)

        self.load_agent()

    def load_agent(self):
        """
        Load the

To explain more on how to create the custom connector:<br>
1. We create a new Class called `MathApplicationConnector`, then inherit the <b>[Connector](https://github.com/aiverify-foundation/moonshot/blob/main/moonshot/src/connectors/connector.py)</b> superclass.

2. We create a new `load_agent` method, where it loads the necessary tools and configuration. This method includes reading the OpenAI API Key, initializes the language model, sets up the tools for solving math problems and answering logic questions. This is the part where we load the agent through Langchain with all the prompt templates and so on.

3. We will initialise the `load_agent` method in the constructor.<br>`self.load_agent()`

4. Every connector will require this method `async def get_response(self, prompt: str) -> ConnectorResponse`.<br>
The purpose of this method is to be responsible of invoking the `self._client` to predict the response of a provided prompt input. <br>
Some common modifications in this function may include adding system prompt or setting specific model parameters.<br>
<br>
Input: prompt (str) - The prompt that requires the prediction. Prompt is from the dataset.<br>
Return: [ConnectorResponse](https://github.com/aiverify-foundation/moonshot/blob/main/moonshot/src/connectors/connector_response.py) - An instance of ConnectorResponse which contains response and context (if required)
<br><br>
<b>ConnectorResponse</b> provides 2 parameters: response and context.<br>
response - this should contain the response from the llm.<br>
context - this should contain the context from the llm, if required.<br>

5. We create a private method `async def _process_response(self, response: Any)` which takes in a response argument.<br>
The purpose of this method is to be responsible of retrieving the output from the llm response.<br>
In this scenario, the llm response will be in this format: `{"input": <prompt>, "output": <response>}`.<br>
We will need to extract the response which is in the `"output"` key, hence `return response["output"]`.

## Create endpoint using custom connector

Now that we have created the custom connector above, we will need a create the endpoint.

To understand more on how endpoint works, you may refer to the `Tutorial 1 - Basic Workflow - Execute a Benchmark`.

In [13]:
endpoint_id = api_create_endpoint(
    "my-custom-math",        # name: Assign a unique name to identify this endpoint later.
    "custom-app",            # connector_type: Specify the connector type for the model you want to evaluate.
    "",                      # uri: Leave blank as the OpenAI library handles the connection.
    "ADD_YOUR_TOKEN_HERE",   # token: Insert your OpenAI API token here.
    1,                       # max_calls_per_second: Set the maximum number of calls allowed per second.
    1,                       # max_concurrency: Set the maximum number of concurrent calls.
    "gpt-4o",                # model: Define the model version to use.
    # params: Include any additional parameters required for this model.
    {
        "timeout": 300,      # timeout: Define the timeout for API calls in seconds.
        "max_attempts": 3,   # max_attempts: Define the max number of retry attempts. 
        "temperature": 0.5,  # temperature: Set the temperature for response variability.
    }  
)
print(f"The newly created endpoint id: {endpoint_id}")

The newly created endpoint id: my-custom-math


### Create dataset and recipe to test my custom endpoint

Let's create a new dataset that has the math questions and answers.

In [14]:
math_dataset = {
    "name": "Math Dataset",
    "description":"Measures whether the model knows how to do math",
    "license": "MIT license",
    "reference": "",
    "examples": [
        {
            "input": "1 + 1 = ?",
            "target": "2"
        },
        {
            "input": "10 * 5 = ?",
            "target": "50"
        },
        {
            "input": "Jane has 5 apples. She gave 3 away. John gave her another 10 apples. How many apples does she have?",
            "target": "12"
        },
        {
            "input": "John has 15 pears. She gave 3 away. John gave her another 10 apples. How many apples does she have?",
            "target": "10"
        },
        {
            "input": "Xiaoming has 3 meat buns. He was given another 10 vegetable buns. How many meat buns does he have?",
            "target": "3"
        }
    ]
}

in_file = f"{moonshot_data_path}/datasets/math-dataset.json"
json.dump(math_dataset, open(in_file, "w+"), indent=4)
if os.path.exists(in_file):
     print(f"Dataset 'math-dataset' has been created.")

Dataset 'math-dataset' has been created.


Let's create a new recipe that allows us to use the math-dataset.

In [15]:
test_recipe = api_create_recipe(
    "Math Questions", # name (mandatory)
    "This recipe is created to test model's ability in answering math questions.", # description (mandatory)
    ["chatbot"], # tags (optional)
    ["capability"], # category (optional)
    ["math-dataset"], # filename of the dataset (mandatory)
    [], # prompt templates (optional)
    ["exactstrmatch"], # metrics (mandatory)
    { # grading scale, optional
        "A": [
            80,
            100
        ],
        "B": [
            60,
            79
        ],
        "C": [
            40,
            59
        ],
        "D": [
            20,
            39
        ],
        "E": [
            0,
            19
        ]
    }
)

print(f"Recipe '{test_recipe}' has been created.")

Recipe 'math-questions' has been created.


### Testing Custom Endpoint with New Recipe

We have created the custom connector, custom connector-endpoint, math-dataset, math-recipe.

Let's run the recipe!

In [16]:
from slugify import slugify
from moonshot.api import api_get_all_run, api_create_runner, api_get_all_runner_name

name = "math recipe" # Indicate the name
recipes = ["math-questions"] # Test one recipe math-questions. You can add more recipes in the list to test as well
endpoints = ["my-custom-math"]  # Test against 1 endpoint, my-custom-math
prompt_selection_percentage = 50 # The percentage number of prompt(s) to run from EACH dataset in the recipe; this refers to 50% of each dataset prompts.

# Optional fields
random_seed = 0   # Default: 0; this allows for randomness in dataset selection when prompt selection percentage are set
system_prompt = ""  # Default: ""; this allows setting the system prompt for the endpoints

# Advanced user - Modify runner processing module and result processing module
# Default: benchmarking and benchmarking-result. Change it to your module name if you have your own runner and/or result module
runner_proc_module = "benchmarking"  # Default: "benchmarking"
result_proc_module = "benchmarking-result"  # Default: "benchmarking-result"

# Run the recipe with the defined endpoint(s)
# If the id exists, it will perform a load on the runner, instead of creating a new runner.
# Using an existing runner allows the new run to possibly use cached results from previous runs, which greatly reduces the run time
slugify_id = slugify(name, lowercase=True)
if slugify_id in api_get_all_runner_name():
    rec_runner = api_load_runner(slugify_id)
else:
    rec_runner = api_create_runner(name, endpoints)

# run_recies is an async function. Currently there is no sync version.
# We will get an existing event loop and execute the run recipes process.
await rec_runner.run_recipes(
    recipes,
    prompt_selection_percentage,
    random_seed,
    system_prompt,
    runner_proc_module,
    result_proc_module,
)
await rec_runner.close()  # Perform a close on the runner to allow proper cleanup.

# Display results
runner_runs = api_get_all_run(rec_runner.id)
result_info = runner_runs[-1].get("results")
if result_info:
    print(json.dumps(result_info, indent=4))
else:
    raise RuntimeError("no run result generated")

2024-12-20 00:03:25,211 [INFO][runner.py::run_recipes(349)] [Runner] math-recipe - Running benchmark recipe run...
2024-12-20 00:03:25,254 [INFO][benchmarking.py::generate(169)] [Benchmarking] Running recipes (['math-questions'])...
2024-12-20 00:03:25,254 [INFO][benchmarking.py::generate(173)] [Benchmarking] Running recipe math-questions... (1/1)
2024-12-20 00:03:25,262 [INFO][connector.py::get_prediction(348)] [Connector ID: my-custom-math] Predicting Prompt Index 3.
2024-12-20 00:03:30,626 [INFO][connector.py::get_prediction(348)] [Connector ID: my-custom-math] Predicting Prompt Index 4.
2024-12-20 00:03:33,267 [INFO][benchmarking.py::generate(203)] [Benchmarking] Run took 8.0126s
2024-12-20 00:03:33,272 [INFO][benchmarking.py::generate(258)] [Benchmarking] Preparing results took 0.0001s
2024-12-20 00:03:33,294 [INFO][benchmarking-result.py::generate(58)] [BenchmarkingResult] Generate results took 0.0205s
2024-12-20 00:03:33,296 [INFO][runner.py::run_recipes(375)] [Runner] math-reci

{
    "metadata": {
        "id": "math-recipe",
        "start_time": "2024-12-20 00:03:25",
        "end_time": "2024-12-20 00:03:33",
        "duration": 8,
        "status": "completed",
        "recipes": [
            "math-questions"
        ],
        "cookbooks": null,
        "endpoints": [
            "my-custom-math"
        ],
        "prompt_selection_percentage": 50,
        "random_seed": 0,
        "system_prompt": ""
    },
    "results": {
        "recipes": [
            {
                "id": "math-questions",
                "details": [
                    {
                        "model_id": "my-custom-math",
                        "dataset_id": "math-dataset",
                        "prompt_template_id": "no-template",
                        "data": [
                            {
                                "prompt": "John has 15 pears. She gave 3 away. John gave her another 10 apples. How many apples does she have?",
                                "